In [1]:
{-# LANGUAGE LambdaCase #-}
{-# LANGUAGE NamedFieldPuns #-}
{-# LANGUAGE ScopedTypeVariables #-}

import Control.Monad.ST  (ST, runST)
import Data.STRef        (STRef, newSTRef, readSTRef, modifySTRef', writeSTRef)
import Control.Monad     (forM_)
import Data.Array        ((!), bounds)
import Data.Array.MArray (newArray, readArray, writeArray)
import Data.Array.ST     (STArray)
import Data.Graph        (Graph, vertices, buildG)

data TarjanEnv s = TarjanEnv
    { index    :: STRef s Int
    , stack    :: STRef s [Int]
    , stackSet :: STArray s Int Bool
    , indices  :: STArray s Int (Maybe Int)
    , lowlinks :: STArray s Int (Maybe Int)
    , output   :: STRef s (Maybe [[Int]])
    , possible :: STRef s Bool
    }

whenM :: Monad m => m Bool -> m () -> m ()
whenM condM block = condM >>= \cond -> if cond then block else return ()

tarjan :: Int -> Graph -> Maybe [[Int]]
tarjan n graph = runST $ do
    tarjanEnv <- TarjanEnv
        <$> newSTRef 0
        <*> newSTRef []
        <*> newArray (0, size) False
        <*> newArray (0, size) Nothing
        <*> newArray (0, size) Nothing
        <*> newSTRef (Just [])
        <*> newSTRef True

    forM_ (vertices graph) $ \v ->
        whenM ((&&)
            <$> ((==) Nothing <$> readArray (indices tarjanEnv) v)
            <*> readSTRef (possible tarjanEnv)) $
                strongConnect n v graph tarjanEnv

    readSTRef (output tarjanEnv)
    where
        size = snd (bounds graph)

strongConnect :: forall s. Int -> Int -> Graph -> TarjanEnv s -> ST s ()
strongConnect n v graph tarjanEnv@TarjanEnv{ index, stack, stackSet, indices, lowlinks, output, possible } = do
    i <- readSTRef index
    writeArray indices  v (Just i)
    writeArray lowlinks v (Just i)
    modifySTRef' index (+1)
    push v

    forM_ (graph ! v) $ \w -> readArray indices w >>= \case
        Nothing -> do
            strongConnect n w graph tarjanEnv
            writeArray lowlinks v =<< (min <$> readArray lowlinks v <*> readArray lowlinks w)
        Just{}  -> whenM (readArray stackSet w) $
            writeArray lowlinks v =<< (min <$> readArray lowlinks v <*> readArray indices  w)

    whenM ((==) <$> readArray lowlinks v <*> readArray indices v) $ do
        scc <- addSCC n v []
        modifySTRef' output $ \sccs -> (:) <$> scc <*> sccs
    where
        addSCC :: Int -> Int -> [Int] -> ST s (Maybe [Int])
        addSCC n v scc = pop >>= \w -> if ((other n w) `elem` scc)
            then writeSTRef possible False >> return Nothing
            else
                let scc' = w:scc
                in if w == v then return (Just scc') else addSCC n v scc'
        push :: Int -> ST s ()
        push e = do
            modifySTRef' stack (e:)
            writeArray stackSet e True
        pop :: ST s Int
        pop = do
            e <- head <$> readSTRef stack
            modifySTRef' stack tail
            writeArray stackSet e False
            return e

denormalise     = subtract
normalise       = (+)
other n v       = 2*n - v
clauses n [u,v] = [(other n u, v), (other n v, u)]

checkSat :: String -> IO Bool
checkSat name = do
    p <- map (map read . words) . lines <$> readFile name
    let pNo    = head $ head p
        pn     = map (map (normalise pNo)) $ tail p
        pGraph = buildG (0,2*pNo) $ concatMap (clauses pNo) pn
    return $ (Nothing /=) $ tarjan pNo pGraph

In [2]:
checkSat "input/2sat1.txt"

True

In [3]:
checkSat "input/2sat2.txt"

False

In [4]:
checkSat "input/2sat3.txt"

True

In [5]:
checkSat "input/2sat4.txt"

True

In [6]:
checkSat "input/2sat5.txt"

False

In [7]:
checkSat "input/2sat6.txt"

False